In [1]:

# Data 전처리
## 숫자 벡터 사전 만들기 
## 데이터 불러오기
## 데이터 x t로 나누기
## 데이터 라벨링=정수로 바꾸기(신경망 안에 벡터 사전이 있을 때) or 벡터화=1차원 배열(신경망 안에 벡터 사진이 없을 때)
### 참고로 우리는 아직 <sos>와 <eos>를 쓰지 않을 겁니다.

# AI 만들기
## AI 학습
### DataLoader에 싣기 (x, t 입력)
### Encoder와 Decoder 구현 하기 (함수 만들기)
### y = F(x) (순전파)
### 손실함수
### 역전파

In [2]:
# 필요한 모듈 import
import os
import sys
sys.path.append(os.path.join(os.path.dirname(""), ".."))
import custom
import numpy as np

# Data 전처리 모듈 : custom과 numpy만으로 가능함
from tqdm import tqdm #자료가 많으니 진행상황은 봅시다

# AI 만들기 모듈
import torch
import torch.nn as nn #클래스 형식의 신경망 함수
import torch.nn.functional as F #함수 형식의 신경망 함수
from torch.utils.data import DataLoader #데이터 batch 나눠주는 역할의 class

In [3]:
## 숫자 벡터 사전 만들기
## ' ',1,2,3,4,5,6,7,8,9,0,+ : 12개
## ' '(pad 문자) 는 0행렬
## 나머지 11개는 원 핫 인코딩

#벡터 값 만들기
vector_list = np.zeros((1,11)) # np.array의 백터 값 저장할 pad값만 저장된 행렬
vector_list = np.append(vector_list, np.eye(11), axis = 0) # 대각선 값은 1이고 나머지는 0인 정사각 행렬
print(vector_list)
print(vector_list.shape)

#숫자-라벨링 사전
num_dict = {}
num_dict[' '] = 0

for i in range(1,10) : #1~9까지 반복문으로 넣기
    num_dict[str(i)] = i
num_dict['0'] = 10
num_dict['+'] = 11
print(num_dict)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
(12, 11)
{' ': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '0': 10, '+': 11}


In [4]:
## 데이터 불러오기
file_name = 'addition.txt'
with open(file_name, mode = "r") as f:
    data = f.readlines()

print(len(data))
print(data[10])

50000
18+8   _26  



In [5]:
## 데이터 x, t로 나누기
data_x = []
data_t = []

for d in data :
    d = d.split("_")
    data_x.append(d[0])
    data_t.append(d[1].replace('\n','')) #개행문자 제거

print("x의 갯수 :",len(data_x))
print("t의 갯수 :",len(data_t))
print("10번째 x :",data_x[10])
print("10번째 t :",data_t[10])
print("10번째 x 길이 :",len(data_x[10]))
print("10번째 t 길이 :",len(data_t[10]))

x의 갯수 : 50000
t의 갯수 : 50000
10번째 x : 18+8   
10번째 t : 26  
10번째 x 길이 : 7
10번째 t 길이 : 4


In [6]:
## 데이터 라벨링=정수로 바꾸기(신경망 안에 벡터 사전이 있을 때) or 벡터화=1차원 배열(신경망 안에 벡터 사진이 없을 때)
## 라벨링 하겠습니다 (= 신경망 안에 벡터 list 넣겠습니다)

for i in range(len(data_x)) :
    temp = data_x[i]
    temp = list(temp) #문장을 문자 하나하나씩 끊어서 list화 #역산 하려면 "".join() 쓰면 됩니다
    vector = custom.word_vectorize(temp, num_dict, 7)
    data_x[i] = vector
data_x = np.array(data_x)

for i in range(len(data_t)) :
    temp = data_t[i]
    temp = list(temp)
    vector = custom.word_vectorize(temp, num_dict, 4)
    data_t[i] = vector
data_t = np.array(data_t)

print(data_x.shape)
print(data_t.shape)


(50000, 7)
(50000, 4)


In [7]:
## AI 학습
### DataLoader에 싣기 (x, t 입력)
device = "cuda" if torch.cuda.is_available() else "cpu"

tensor_x = torch.tensor(data_x, dtype = torch.long, device = device)
tensor_t = torch.tensor(data_t, dtype = torch.long, device = device)

### 앞의 40000번째 까지는 train, 뒤의 10000개는 test(val)
s = 40000
train_zip = list(zip(tensor_x[:s], tensor_t[:s])) 
test_zip = list(zip(tensor_x[s:], tensor_t[s:]))

train_dataloader = DataLoader(train_zip, batch_size=100, shuffle=True)
test_dataloader = DataLoader(test_zip, batch_size=1000, shuffle=False)
print(len(train_dataloader))
print(len(test_dataloader))

400
10


In [8]:
### Encoder와 Decoder 구현 하기 (함수 만들기)

from add_ai1 import Encoder
from add_ai1 import Decoder
from add_ai1 import Encoder_n_Decoder

In [9]:
encoder_file_name = 'data/encoder1.pt'
decoder_file_name = 'data/decoder1.pt'

tensor_vector = torch.tensor(vector_list, dtype = torch.float)
encoder = Encoder(tensor_vector).to(device)
decoder = Decoder(tensor_vector, 4).to(device)
loss_func = nn.CrossEntropyLoss()
encoder_optim = torch.optim.Adam(encoder.parameters(), lr = 0.015)
decoder_optim = torch.optim.Adam(decoder.parameters(), lr = 0.025)

epoch = 100
prev_acc = 0
cnt = 0

for e in range(epoch) :
    loss_sum = 0
    encoder.train() #dropout 켜주기
    decoder.train()
    for x, t in train_dataloader :
### y = F(x) (순전파)
        y, h = encoder(x)
        y, _, _ = decoder(y, h)
### 손실함수
        loss = loss_func(y.reshape(-1, y.shape[-1]) , t.reshape(-1)) #3차원을 2차원으로 펴주기
        loss_sum += loss.item()
### 역전파
        loss.backward()
        decoder_optim.step()
        encoder_optim.step()
        decoder_optim.zero_grad()
        encoder_optim.zero_grad()
    loss_sum /= len(train_dataloader) #평균 구하기
### 중간 점검
    correct = 0
    total = 0
    encoder.eval() #dropout 꺼주기
    decoder.eval()
    for x, t in test_dataloader :
        with torch.no_grad() :
            y, h = encoder(x)
            y, _, _ = decoder(y, h)
            correct += (y.argmax(dim = -1) == t).sum().item()
        total += len(x) * 4
    acc = correct / total
### earlystopper
    if acc <= prev_acc :
        cnt += 1
    else :
        cnt = 0
        prev_acc = acc
        torch.save(encoder, encoder_file_name) #중간 저장
        torch.save(decoder, decoder_file_name)
    print(f"epoch {e+1} | loss {loss_sum} | acc {acc} | cnt {cnt}")
    if cnt >= 5 :
        print("train stopped")
        break

epoch 1 | loss 1.8145816707611084 | acc 0.36995 | cnt 0
epoch 2 | loss 1.5578071987628936 | acc 0.38945 | cnt 0
epoch 3 | loss 1.4748249331116676 | acc 0.4289 | cnt 0
epoch 4 | loss 1.4331014129519462 | acc 0.440225 | cnt 0
epoch 5 | loss 1.4171544337272644 | acc 0.433875 | cnt 1
epoch 6 | loss 1.399076808989048 | acc 0.46225 | cnt 0
epoch 7 | loss 1.396468303501606 | acc 0.440725 | cnt 1
epoch 8 | loss 1.3525731045007705 | acc 0.467675 | cnt 0
epoch 9 | loss 1.3431741312146186 | acc 0.48485 | cnt 0
epoch 10 | loss 1.3437073037028313 | acc 0.4868 | cnt 0
epoch 11 | loss 1.337771989107132 | acc 0.485025 | cnt 1
epoch 12 | loss 1.3234265261888505 | acc 0.4861 | cnt 2
epoch 13 | loss 1.3119307807087899 | acc 0.483375 | cnt 3
epoch 14 | loss 1.318671189546585 | acc 0.48135 | cnt 4
epoch 15 | loss 1.3190260675549508 | acc 0.49635 | cnt 0
epoch 16 | loss 1.3133768835663795 | acc 0.492275 | cnt 1
epoch 17 | loss 1.3026915562152863 | acc 0.4871 | cnt 2
epoch 18 | loss 1.3044801476597785 | acc 

In [10]:
# onnx로 변환하기 쉽게 하기 위해 다시 불러와서 cpu로 변환 한 다음에 하나로 묶는 작업

encoder = torch.load(encoder_file_name, weights_only=False, map_location='cpu') #cpu로 불러오기 
decoder = torch.load(decoder_file_name, weights_only=False, map_location='cpu')

F = Encoder_n_Decoder(encoder, decoder)
print(F)
print(F.state_dict()['encoder.embedding.weight']) #.state_dict() : 신경망 내의 W, b 값 불러오는 함수



Encoder_n_Decoder(
  (encoder): Encoder(
    (embedding): Embedding(12, 11, padding_idx=0)
    (rnn): RNN(11, 11, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(12, 11, padding_idx=0)
    (rnn): RNN(11, 11, batch_first=True)
    (f): Linear(in_features=11, out_features=12, bias=True)
  )
)
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])


In [11]:
# 묶은 신경망을 다시 저장

torch.save(F, "data/add_ai1.pt")